# Azure Machine Learning による機械学習プロセス - デプロイ編

## アジェンダ
### A. 学習編
- ワークスペース (Workspace) への接続
- データセット (Datasets) の登録
- 環境 (Environments) の登録
- コンピューティングクラスター (Compute Clusters) の作成
- モデル学習の実行と実験 (Run & Experiments)
- モデル登録 (Models)

### **B. デプロイ編 (本ノートブック)**
- ワークスペース (Workspace) への接続
- 推論環境の作成 (Deployment)
- エンドポイントの利用 (Endpoint)

## 事前設定
- 本ノートブックは Azure Machine Learning の Compute Instance を利用することを想定しています。
- 予め [train-notebook.ipynb](train-notebook.ipynb) を実行して、モデル学習のジョブ実行と学習済みモデルの登録が完了していること。
- カーネルは `azureml_py36 (Python 3.6.9)` を選択ください。

<br>

## ワークスペース (Workspace) への接続
作業環境から Azure Machine Learning Workspace へ接続を行います。

#### Azure Machine Learning Studio
[ml.azureml.com](ml.azurem.com) にアクセスします。Python SDK を中心に作業される場合にも Azure Machine Learning Studio を併用することが多いです。

<img src="../docs/images/azureml-workspace.png" width=500>


#### Python SDK での手順
クライアント環境の Python 環境にインストールした Azure ML Python SDK を用いて Azure Machine Learning Workspace に接続します。

In [ ]:
# Compute Instance を利用する場合
from azureml.core import Workspace
ws = Workspace.from_config()

In [ ]:
# # その他の任意のクライアント環境を利用する場合
# ws = Workspace.get(
#     name='name',
#     subscription_id='subscription_id',
#     resource_group='resource_group',
# )

<br>

## 推論環境の作成 (Deployment)
下記の情報を利用してモデルをデプロイし、推論環境を作成します。
- 登録済みのモデル (Model)
- 推論環境で稼働する環境 (Environments)
- 推論スクリプト : _score.py_

#### Python SDK での手順

In [ ]:
from azureml.core.environment import Environment
from azureml.core import Model
from azureml.core.webservice import LocalWebservice, AciWebservice
from azureml.core.model import InferenceConfig

##### 登録済みのモデル (Models)

In [ ]:
model = Model(ws, "lgb-model")

##### 推論環境で稼働する環境 (Environments)

In [ ]:
env = Environment.get(ws, "lightgbm-python-env")
env.inferencing_stack_version = 'latest'

##### 推論スクリプト _score.py_
`script` フォルダに予め作成済みです。

```python

import os
import json
import numpy as np
import pandas as pd
import lightgbm as lgb
from helper import data_preprocess



def init():
    global bst
    model_root = os.getenv("AZUREML_MODEL_DIR")
    # 学習済みモデルを含むフォルダ名の指定
    lgbm_model_folder = "model"
    bst = lgb.Booster(
        model_file=os.path.join(model_root, lgbm_model_folder, "model.lgb")
    )

def run(raw_data):
    categorical_cols = ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']
    float_cols = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
    columns = bst.feature_name()
    data = np.array(json.loads(raw_data)["data"])
    test_df_original = pd.DataFrame(data=data, columns=columns)
    test_df = data_preprocess(test_df_original, categorical_cols, float_cols)
    # 予測値の生成
    out = bst.predict(test_df)
    return out.tolist()

````

コードや環境 (Environments) の情報を設定します。

In [ ]:
inference_config = InferenceConfig(entry_script="score.py", source_directory="script", environment=env)

最初にローカル環境にデプロイをします。

In [ ]:
localconfig = LocalWebservice.deploy_configuration(port=8890)
local_service_name = "mylocalmodel"
local_service = Model.deploy(
    workspace=ws,
    name=local_service_name,
    models=[model],
    inference_config=inference_config,
    deployment_config=localconfig,
    overwrite=True
)

In [ ]:
local_service.wait_fordeployment(show_output=True)

テストデータを入力して予測値を算出します。

In [ ]:
import json

# テストデータ
data =  {
            "data": [[
                2,
                "Kvillner, Mr. Johan Henrik Johannesson",
                "male",
                31,
                0,
                0,
                "C.A. 18723",
                10.5,
                "",
                "S"
            ]]
        }

test_sample = json.dumps(data)
test_sample = str.encode(test_sample, encoding='utf8')

prediction = local_service.run(input_data=test_sample)
print(prediction)

<br>

次に Azure Container Instance へデプロイをします。

In [ ]:
aciconfig = AciWebservice.deploy_configuration(auth_enabled=True)

Azure Container Instance にモデルをデプロイします。<br>
なお、`service_name` は文字から始まる 3 以上 32 小文字・数字・記号 (ダッシュのみ)で記載ください。


In [ ]:
service_name = "lgb-aci3"  # グループで作業している場合は、名前を変更してください
service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[model],
    inference_config=inference_config,
    deployment_config=aciconfig,
    overwrite=True
)

In [ ]:
service.wait_for_deployment(show_output=True)

In [ ]:
service.get_logs()

Azure Machine Learning Studio にて正常に登録されていることを確認します。<br>
<img src="../docs/images/azureml-deployment1.png" width=500><br>



<br>

## エンドポイントの利用 (Endpoint)
推論環境にテストデータをインプットして、デプロイした機械学習モデルから予測値を算出します。

#### Python SDK での手順

In [ ]:
import urllib.request
import json
import os
import ssl

# テストデータ
data =  {
            "data": [[
                2,
                "Kvillner, Mr. Johan Henrik Johannesson",
                "male",
                31,
                0,
                0,
                "C.A. 18723",
                10.5,
                "",
                "S"
            ]]
        }
body = str.encode(json.dumps(data), encoding='utf8')

In [ ]:
url = service.scoring_uri
key, _ = service.get_keys()
headers = {'Content-Type':'application/json'}
headers["Authorization"] = f"Bearer {key}"
req = urllib.request.Request(url, body, headers)

In [ ]:
try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))
